In [2]:
#step1 Reading file and understand data
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('salary_prediction').getOrCreate()
df=spark.read.csv(r"C:\Users\Vamsi\Downloads\test2.csv",header=True,inferSchema=True)
df=df.na.drop()
df.show()
df.describe().show()
df.printSchema()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+

+-------+------+------------------+-----------------+------------------+
|summary|  Name|               age|       Experience|            Salary|
+-------+------+------------------+-----------------+------------------+
|  count|     6|                 6|                6|                 6|
|   mean|  NULL|26.333333333333332|4.666666666666667|21333.333333333332|
| stddev|  NULL| 4.179314138308661|3.559026084010437| 5354.126134736337|
|    min|Harsha|                21|                1|             15000|
|    max| Sunny|                31|               10|             30000|
+-------+------+------------------+-----------------+------------------+

r

In [3]:
#Combine multiple features into single feature
from pyspark.ml.feature import VectorAssembler
feacturecols=['age', 'Experience','Salary']
assembler=VectorAssembler(inputCols=feacturecols,outputCol='features')
final_data=assembler.transform(df)
final_data.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|           features|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|[31.0,10.0,30000.0]|
|Sudhanshu| 30|         8| 25000| [30.0,8.0,25000.0]|
|    Sunny| 29|         4| 20000| [29.0,4.0,20000.0]|
|     Paul| 24|         3| 20000| [24.0,3.0,20000.0]|
|   Harsha| 21|         1| 15000| [21.0,1.0,15000.0]|
|  Shubham| 23|         2| 18000| [23.0,2.0,18000.0]|
+---------+---+----------+------+-------------------+



In [4]:
#Split the data into two 1)train_data 2)test_data
train_data, test_data = final_data.randomSplit([0.75,0.25])
train_data.count()
test_data.count()

1

In [5]:
#Train the model
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='features', labelCol='Salary')
lr_model=lr.fit(train_data)

In [6]:
#Evaluate the model
print('coefficients:',lr_model.coefficients)
print('Intercept:',lr_model.intercept)
test_result=lr_model.evaluate(test_data)
print("RMSE:", test_result.rootMeanSquaredError)
print("R2 Score:", test_result.r2)

coefficients: [6.492514080924944e-14,2.570179603450886e-12,0.999999999999998]
Intercept: 2.8091636523404443e-11
RMSE: 0.0
R2 Score: nan


In [8]:
#Make predictions
prediction=lr_model.transform(test_data)
prediction.show()

+-----+---+----------+------+-------------------+----------+
| Name|age|Experience|Salary|           features|prediction|
+-----+---+----------+------+-------------------+----------+
|Krish| 31|        10| 30000|[31.0,10.0,30000.0]|   30000.0|
+-----+---+----------+------+-------------------+----------+

